Set this name to the base name of the notebook you want to run on a GPU.

In [ ]:
name = "CudaExample"

In [ ]:
from IPython.core.magic import register_line_cell_magic
import asyncio
from pathlib import Path

This is a quick magic command that behaves exactly like `%%writefile` except with variable templating

In [ ]:
@register_line_cell_magic
def writetemplate(line, cell):
    print("(Over)writing", line)
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

And, a quick utility to watch the file until it signals that it is done:

In [ ]:
async def watch_file(filename):
    filename = Path(filename)
    while not filename.exists():
        await asyncio.sleep(.5)
    with open(filename) as f:
        while True:
            r = f.readline()
            if "[ADRIOT-DONE]" in r:
                break
            elif r == '':
                await asyncio.sleep(.5)
            else:
                print(r, end='')
                
    print("Done!")

> Note:
> 
> In Python 3.8 the loop could be written:
> ```python
> while "[ADRIOT-DONE]" not in (r := f.readline()):
    if r == '':
        await asyncio.sleep(.5)
    else:
        print(r, end='')
> ```
>
> You can also write this syncronously by replacing `asyncio` with `time` and removing `async`/`await` everywhere, since we don't interleave it with anything else (but that's much less fun).

In [ ]:
%%writetemplate {name}.sbatch
#!/bin/bash
# GPU job

#SBATCH --job-name={name}        # create a short name for your job
#SBATCH -o {name}.out            # Name of stdout output file (%j expands to jobId)
#SBATCH --nodes=1                # node count
#SBATCH --ntasks=1               # total number of tasks across all nodes
#SBATCH --cpus-per-task=1        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --gres=gpu               # number of gpus per node
#SBATCH --mem=1G                 # total memory (RAM) per node
#SBATCH --time=00:05:00          # total run time limit (HH:MM:SS)

module purge
module load course/pygpu/default

time jupyter nbconvert --to html --execute --ExecutePreprocessor.timeout=60 {name}.ipynb

echo
echo "[ADRIOT-DONE]"

`:tesla_v100:1`

We remove the old run output and output file if they are present.

In [ ]:
!rm {name}.out {name}.html

Time to submit the run!

In [ ]:
!sbatch {name}.sbatch

And now let's watch the output file:

In [ ]:
await watch_file(f'{name}.out');

Open the new nbconvert notebook to see the output!